# Optimization

Tests to determine memory allocation and performance of varaious functions, looks for bottle necks. 

## Quadrature

### Updating quadrature scheme

In [139]:
## initialize quadrature
include("MvGaussHermite.jl")
m = 5
Quad = MvGaussHermite.init_mutable(m,[0.0,0.0],[1.0 0; 0 1.0],10^-4)

# test updating 
MvGaussHermite.update_old!(Quad, [1.0,0.0],[1.0 0.5; 0.5 1.0])
@time for i in 1:10 MvGaussHermite.update_old!(Quad, [1.0,0.0],[1.0 0.5; 0.5 1.0]) end


  0.000226 seconds (1.76 k allocations: 157.500 KiB)


In [140]:
Quad = MvGaussHermite.init_mutable(m,[0.0,0.0],[1.0 0; 0 1.0],10^-4)
MvGaussHermite.update!(Quad, [1.0,0.0],[1.0 0.5; 0.5 1.0])
@time for i in 1:10 MvGaussHermite.update!(Quad, [1.0,0.0],[1.0 0.5; 0.5 1.0])end


  0.000238 seconds (1.22 k allocations: 118.594 KiB)


### Evaluating quadrature

In [155]:
include("MvGaussHermite.jl")
m = 10
f  = x -> sin(x[1]*x[2])
Quad = MvGaussHermite.init_mutable(m,[0.0,0.0],[1.0 0; 0 1.0],10^-4)
MvGaussHermite.expected_value(f, Quad)
@time MvGaussHermite.expected_value(f, Quad)

  0.000034 seconds (16 allocations: 1.938 KiB)


1.6263032587282567e-18

## Updating beleif states


In [16]:
include("MvGaussHermite.jl")
include("BellmanOpperators.jl")
include("Examples.jl")
# set example functions for test s
pars = Examples.unknown_growth_rate_pars

T_! = (x,f) -> Examples.unknown_growth_rate_T!(x,f,pars)
T_ = (x,f) -> Examples.unknown_growth_rate_T(x,f,pars)
R = (x,f,obs) -> Examples.unknown_growth_rate_R(x,f,obs,pars)
R_obs = (x,f) -> Examples.unknown_growth_rate_R_obs(x,f,pars)
Sigma_N = Examples.Sigma_N
H = (x,a,obs) -> Examples.H * x
Sigma_O = (a,obs) -> reshape(Examples.Sigma_O(obs),1,1)
delta = 0.95
actions = Examples.unknown_growth_actions
observations = Examples.unknown_growth_observations

x_hat = [0.0,0.0]
x_cov = [1.0 0; 0 1.0]

m = 6
f  = x -> sin(x[1]*x[2])
Quad = MvGaussHermite.init_mutable(m,[0.0,0.0],[1.0 0; 0 1.0],10^-4)

print(" ")

### Test observaiton model update
The take away here is to use a linear observaiton model where ever possible. 

In [44]:
H = x -> Examples.H * x
BellmanOpperators.propogate_observation_model(x_hat,x_cov,H,Quad)
MvGaussHermite.update!(Quad, x_hat, x_cov)
@time BellmanOpperators.propogate_observation_model(x_hat,x_cov,H,Quad)
@time MvGaussHermite.update!(Quad, x_hat, x_cov)

  0.000090 seconds (453 allocations: 42.781 KiB)
  0.000040 seconds (145 allocations: 13.922 KiB)


2×2 Matrix{Float64}:
 1.0  0.0
 0.0  1.0

In [45]:
H = Examples.H 
BellmanOpperators.propogate_observation_model(x_hat,x_cov,H)
@time BellmanOpperators.propogate_observation_model(x_hat,x_cov,H)

  0.000011 seconds (4 allocations: 320 bytes)


([0.0], [1.0])

### Test bellman opperator for bottle necks 
 The bottle neck is very clearly evaluating the value function

In [72]:
include("MvGaussHermite.jl")
include("BellmanOpperators.jl")
include("POMDPs.jl")
# initianize POMDP object
pars = Examples.unknown_growth_rate_pars
T_! = (x,f) -> Examples.unknown_growth_rate_T!(x,f,pars)
T_ = (x,f) -> Examples.unknown_growth_rate_T(x,f,pars)
R = (x,f,obs) -> 0.5*exp(x[1]+f[1]) - 1/obs[1]
R_obs = (x,f) -> Examples.unknown_growth_rate_R_obs(x,f,pars)
Sigma_N = Examples.Sigma_N
H = (x,a,obs) -> Examples.H * x
Sigma_O = (a,obs) -> reshape(Examples.Sigma_O(obs),1,1)
delta = 0.95
unknownGrowthRate = POMDPs.init(T_!,T_,R,R_obs,H,Sigma_N,Sigma_O,delta, [[0.05],[0.1],[0.15]], [[0.05],[1.0]])


# state 
act = [0.15]
obs = [1.0]

# value function 
function test1!(mu)
    return 0.1*mu[2] * mu[1]^2
end 


function test2!(mu, cov)
    v = test1!(mu)
    return v - 0.1*cov[1,1] -0.1*cov[2,2] + 0.05*cov[2,1] +0.75*cov[1,1]*mu[1]
end 

m1 = 20
m2 = 7
Vf = ValueFunctions.init_adjGausianBeleifsInterp(m1, m2, [-5.0,-5.0],[5.0,5.0])

v1 = 1.0*zeros(m1^2)
v2 = 1.0*zeros(m2^5)
v_mu = broadcast(i->test1!(Vf.baseValue.grid[i]), 1:m1^2)
v_dsn = broadcast(i->test2!(Vf.uncertantyAdjustment.nodes[i][1],Vf.uncertantyAdjustment.nodes[i][2]), 1:m2^5)

ValueFunctions.update_base!(Vf,v_mu)
ValueFunctions.update_adjustment!(Vf, v_dsn)

V = (x,a) -> p(x)
x_hat = [0.4, 0.0]
x_cov = [0.2 0.0; 0.0 0.1]
s = (x_hat,x_cov)


data = BellmanOpperators.init_bellmanIntermidiate(2, 1, 5, 5)
print(" ")

In [70]:
function f1(s, a, obs, data, POMDP, V)
    data.x_hat, data.x_cov = s
    tu = KalmanFilters.time_update(data.x_hat, data.x_cov, x ->POMDP.T(x,a),  POMDP.Sigma_N)
    data.x_hat, data.x_cov = KalmanFilters.get_state(tu), KalmanFilters.get_covariance(tu)
    
    #observaton quadrature nodes 
    H = x -> POMDP.H(x,a,obs) # define measurment function # allocation? 
    data.y_hat, data.y_cov = BellmanOpperators.propogate_observation_model(data.x_hat,data.x_cov,H,data.Quad_x) # allocation 
    data.y_cov .+= POMDP.Sigma_O(a,obs)
    MvGaussHermite.update!(data.Quad_y,data.y_hat,data.y_cov)
    data.new_states_mat = broadcast(y -> BellmanOpperators.new_state(y, data.x_hat,data.x_cov, H, POMDP.Sigma_O(a,obs)), data.Quad_y.nodes)
    return 0
end 

f1(s,act,obs, data, unknownGrowthRate, V)
@time f1(s,act,obs, data, unknownGrowthRate, V)

  0.000248 seconds (781 allocations: 68.203 KiB)


0

In [73]:
function f2(s, a, obs, data, POMDP, V)
    data.x_hat, data.x_cov = s
    tu = KalmanFilters.time_update(data.x_hat, data.x_cov, x ->POMDP.T(x,a),  POMDP.Sigma_N)
    data.x_hat, data.x_cov = KalmanFilters.get_state(tu), KalmanFilters.get_covariance(tu)
    
    #observaton quadrature nodes 
    H = x -> POMDP.H(x,a,obs) # define measurment function # allocation? 
    data.y_hat, data.y_cov = BellmanOpperators.propogate_observation_model(data.x_hat,data.x_cov,H,data.Quad_x) # allocation 
    data.y_cov .+= POMDP.Sigma_O(a,obs)
    MvGaussHermite.update!(data.Quad_y,data.y_hat,data.y_cov)
    data.new_states_mat = broadcast(y -> BellmanOpperators.new_state(y, data.x_hat,data.x_cov, H, POMDP.Sigma_O(a,obs)), data.Quad_y.nodes)
    data.vals = broadcast(x -> V(data.z,x), data.new_states_mat)
    sum(data.vals .* data.Quad_y.weights)
    return 0
end 

f2(s,act,obs, data, unknownGrowthRate, V)
@time f2(s,act,obs, data, unknownGrowthRate, V)
    

  0.000712 seconds (8.00 k allocations: 480.609 KiB)


0

### Test Value functions

In [68]:
include("ValueFunctions.jl")

Main.ValueFunctions

In [85]:
# value function 
function test1!(mu)
    return 0.1*mu[2] * mu[1]^2
end 


function test2!(mu, cov)
    v = test1!(mu)
    return v - 0.1*cov[1,1] -0.1*cov[2,2] + 0.05*cov[2,1] +0.75*cov[1,1]*mu[1]
end 

m1 = 10
m2 = 6
Vf = ValueFunctions.init_adjGausianBeleifsInterp(m1, m2, [-5.0,-5.0],[5.0,5.0])

v1 = 1.0*zeros(m1^2)
v2 = 1.0*zeros(m2^5)
v_mu = broadcast(i->test1!(Vf.baseValue.grid[i]), 1:m1^2)
v_dsn = broadcast(i->test2!(Vf.uncertantyAdjustment.nodes[i][1],Vf.uncertantyAdjustment.nodes[i][2]), 1:m2^5)

ValueFunctions.update_base!(Vf,v_mu)
ValueFunctions.update_adjustment!(Vf, v_dsn)

z = 1.0*zeros(5)
x = ([0.0,0.0],[1.0 0; 0 1.0])
Vf.baseValue(x[1])
@time Vf.baseValue(x[1])

Vf.uncertantyAdjustment(z,x)
@time Vf.uncertantyAdjustment(z,x)

  0.000066 seconds (248 allocations: 11.875 KiB)
  0.000114 seconds (1.44 k allocations: 82.000 KiB)


-0.19976206848347985

### Evaluating the uncertiaty adjustment in the value function is the primary bottle neck in the bellman opperator

Two steps to optimize this process: 
1. Optimize evaluation of chebyshev polynomials in ValueFunctions.jl
2. Down sample the value function i.e use a low order aproximation if it apears to be apropreate.   

In [133]:
include("ValueFunctions.jl")
a = [-1.0,-1.]
b = [1.0,1.]
m = 100
p = ValueFunctions.init_interpolation(a,b,m)
x = [0.0,0.0]
# value function 
function test1!(mu)
    return 0.1*mu[2] * mu[1]^2
end 

v_mu = broadcast(i->test1!(p.grid[i]), 1:length(p.grid))

ValueFunctions.update_interpolation!(p,reshape(v_mu,p.m,p.m))
p(x)
@time p(x)


  0.000682 seconds (20.14 k allocations: 799.438 KiB)


3.228315458645783e-16

In [134]:
include("utils.jl")
utils.T_alpha_i(p.alpha[1],x)
@time utils.T_alpha_i(p.alpha[5000],x)

  0.000008 seconds (2 allocations: 112 bytes)


6.123233995736766e-17

In [135]:
utils.T_alpha(x,p.alpha, p.coeficents)
@time utils.T_alpha(x,p.alpha, p.coeficents)

  0.000629 seconds (20.12 k allocations: 798.312 KiB)


3.228315458645783e-16

In [152]:
include("utils.jl")
v = [0.0]
utils.T_alpha_i!(v,p.alpha[10],x)
@time utils.T_alpha_i!(v,p.alpha[10],x)

  0.000010 seconds (1 allocation: 96 bytes)


1-element Vector{Float64}:
 5.51091059616309e-16

In [155]:
v = broadcast(i ->[0.0], 1:length(p.alpha))
utils.T_alpha!(v,x,p.alpha, p.coeficents)
@time utils.T_alpha!(v,x,p.alpha, p.coeficents)

  0.000855 seconds (15.45 k allocations: 1.454 MiB)


1-element Vector{Float64}:
 3.2283154586457827e-16

In [162]:
v = broadcast(i ->[0.0], 1:length(p.alpha))
g1(x) = for i in 1:1:length(p.alpha) utils.T_alpha_i!(v[i],p.alpha[i],x)end
g1(x)
@time g1(x)

  0.000984 seconds (19.58 k allocations: 788.922 KiB)


In [163]:

g2(x) = broadcast(i -> utils.T_alpha_i!(v[i],p.alpha[i],x), 1:length(p.alpha))
g2(1)
@time g2(1)


  0.000588 seconds (13.95 k allocations: 259.000 KiB)


5151-element Vector{Vector{Float64}}:
 [1.0]
 [1.0]
 [1.0]
 [1.0]
 [1.0]
 [1.0]
 [1.0]
 [1.0]
 [1.0]
 [1.0]
 [1.0]
 [1.0]
 [1.0]
 ⋮
 [1.0]
 [1.0]
 [1.0]
 [1.0]
 [1.0]
 [1.0]
 [1.0]
 [1.0]
 [1.0]
 [1.0]
 [1.0]
 [1.0]

In [ ]:
# the nth degree chebyshev polynomial
# evaluated at x
function T!(x,alpha)
    prod(cos.(alpha_i.*acos.(x))
end 

T1